<a href="https://colab.research.google.com/github/Rony-pat/Clases/blob/main/Tarea_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd


df_2016 = pd.read_csv('emisiones-2016.csv', delimiter=';')
df_2017 = pd.read_csv('emisiones-2017.csv', delimiter=';')
df_2018 = pd.read_csv('emisiones-2018.csv', delimiter=';')


def procesar_dataframe(df):

    columnas_dias = [f'D{str(i).zfill(2)}' for i in range(1, 32)]  # D01, D02, ..., D31
    columnas_necesarias = ['ESTACION', 'MAGNITUD', 'ANO', 'MES'] + columnas_dias
    df_filtrado = df[columnas_necesarias]


    df_reestructurado = pd.melt(df_filtrado,
                                id_vars=['ESTACION', 'MAGNITUD', 'ANO', 'MES'],
                                value_vars=columnas_dias,
                                var_name='DIA',
                                value_name='VALOR')


    df_reestructurado['DIA'] = df_reestructurado['DIA'].str.extract('(\d+)').astype(int)

    return df_reestructurado


df_2016_procesado = procesar_dataframe(df_2016)
df_2017_procesado = procesar_dataframe(df_2017)
df_2018_procesado = procesar_dataframe(df_2018)


print(df_2016_procesado.head())



   ESTACION  MAGNITUD   ANO  MES  DIA  VALOR
0         4         1  2016    1    1    8.0
1         4         1  2016    2    1   12.0
2         4         1  2016    3    1   11.0
3         4         1  2016    4    1    8.0
4         4         1  2016    5    1    7.0


In [25]:
import numpy as np
from datetime import datetime


def procesar_dataframe_con_fecha(df):

    df_reestructurado = procesar_dataframe(df)

    df_reestructurado['FECHA'] = pd.to_datetime(df_reestructurado[['ANO', 'MES', 'DIA']].rename(columns={'ANO':'year', 'MES':'month', 'DIA':'day'}), format='%Y%m%d', errors='coerce')

    df_reestructurado = df_reestructurado[~df_reestructurado['FECHA'].isna()]

    df_reestructurado = df_reestructurado.sort_values(by=['ESTACION', 'FECHA'])

    return df_reestructurado

df_2016_procesado_con_fecha = procesar_dataframe_con_fecha(df_2016)
df_2017_procesado_con_fecha = procesar_dataframe_con_fecha(df_2017)
df_2018_procesado_con_fecha = procesar_dataframe_con_fecha(df_2018)

print(df_2016_procesado_con_fecha.head())

    ESTACION  MAGNITUD   ANO  MES  DIA  VALOR      FECHA
0          4         1  2016    1    1    8.0 2016-01-01
12         4         6  2016    1    1    0.4 2016-01-01
24         4         7  2016    1    1   28.0 2016-01-01
36         4         8  2016    1    1   40.0 2016-01-01
48         4        12  2016    1    1   83.0 2016-01-01


In [26]:
def mostrar_estaciones_y_contaminantes(df):
    estaciones = df['ESTACION'].unique()
    contaminantes = df['MAGNITUD'].unique()
    print("Estaciones disponibles:", estaciones)
    print("Contaminantes disponibles:", contaminantes)
mostrar_estaciones_y_contaminantes(df_2016_procesado_con_fecha)

def emisiones_por_estacion_y_contaminante(df, estacion, contaminante, fecha_inicio, fecha_fin):
    df_filtrado = df[(df['ESTACION'] == estacion) & (df['MAGNITUD'] == contaminante)]
    df_filtrado = df_filtrado[(df_filtrado['FECHA'] >= fecha_inicio) & (df_filtrado['FECHA'] <= fecha_fin)]
    return df_filtrado[['FECHA', 'VALOR']].set_index('FECHA')
fecha_inicio = '2016-01-01'
fecha_fin = '2016-12-31'
estacion = 4
contaminante = 1
emisiones = emisiones_por_estacion_y_contaminante(df_2016_procesado_con_fecha, estacion, contaminante, fecha_inicio, fecha_fin)
print(emisiones)


Estaciones disponibles: [ 4  8 11 16 17 18 24 27 35 36 38 39 40 47 48 49 50 54 55 56 57 58 59 60]
Contaminantes disponibles: [ 1  6  7  8 12  9 10 14 20 30 35 42 43 44]
            VALOR
FECHA            
2016-01-01    8.0
2016-01-02    7.0
2016-01-03    6.0
2016-01-04    6.0
2016-01-05    7.0
...           ...
2016-12-27   17.0
2016-12-28   13.0
2016-12-29   13.0
2016-12-30   12.0
2016-12-31    7.0

[366 rows x 1 columns]


In [28]:
def resumen_descriptivo_por_contaminante(df):
    resumen = df.groupby('MAGNITUD')['VALOR'].describe()
    return resumen

print("Resumen descriptivo por contaminante (2016):")
print(resumen_descriptivo_por_contaminante(df_2016_procesado_con_fecha))

def resumen_descriptivo_por_estacion_y_contaminante(df):
    resumen = df.groupby(['ESTACION', 'MAGNITUD'])['VALOR'].describe()
    return resumen

print("\nResumen descriptivo por estación y contaminante (2016):")
print(resumen_descriptivo_por_estacion_y_contaminante(df_2016_procesado_con_fecha))

def resumen_por_estacion_y_contaminante(df, estacion, contaminante):
    df_filtrado = df[(df['ESTACION'] == estacion) & (df['MAGNITUD'] == contaminante)]
    resumen = df_filtrado['VALOR'].describe()
    return resumen

estacion = 4
contaminante = 1

print(f"\nResumen descriptivo para la estación {estacion} y el contaminante {contaminante}:")
print(resumen_por_estacion_y_contaminante(df_2016_procesado_con_fecha, estacion, contaminante))


Resumen descriptivo por contaminante (2016):
           count       mean         std   min    25%    50%    75%       max
MAGNITUD                                                                    
1         3660.0   7.656831    6.186077  0.00   4.00   7.00  10.00    234.00
6         3660.0   0.350082    0.160851  0.00   0.30   0.30   0.40      1.40
7         8784.0  24.737591  265.477871  0.00   5.00  10.00  26.00  24742.00
8         8784.0  38.301799   19.076459  0.00  24.00  36.00  50.00    142.00
9         2196.0  10.398907    6.380443  0.00   6.00   9.00  13.00     84.00
10        4392.0  19.023452   15.909335  0.00  10.00  16.00  24.00    257.00
12        8784.0  71.910291   62.169066  0.00  32.00  53.00  90.00    586.00
14        5124.0  48.646175   24.018242  0.00  31.00  50.00  66.25    122.00
20        2196.0   2.985974    6.247159  0.00   1.00   1.90   3.50    185.50
30        2196.0   0.653734    0.749189  0.00   0.20   0.40   0.80     15.10
35        2196.0   0.430965    

In [29]:
def emisiones_medias_mensuales(df, contaminante, ano):
    df_filtrado = df[(df['ANO'] == ano) & (df['MAGNITUD'] == contaminante)]
    emisiones_medias = df_filtrado.groupby('MES')['VALOR'].mean()
    return emisiones_medias

contaminante = 1
ano = 2016

print(f"Emisiones medias mensuales para el contaminante {contaminante} en {ano}:")
print(emisiones_medias_mensuales(df_2016_procesado_con_fecha, contaminante, ano))

def medidas_mensuales_por_estacion(df, estacion):
    df_filtrado = df[df['ESTACION'] == estacion]
    medidas_mensuales = df_filtrado.groupby(['MES', 'MAGNITUD'])['VALOR'].mean().unstack()
    return medidas_mensuales

estacion = 4

print(f"\nMedidas mensuales de los distintos contaminantes para la estación {estacion}:")
print(medidas_mensuales_por_estacion(df_2016_procesado_con_fecha, estacion))


Emisiones medias mensuales para el contaminante 1 en 2016:
MES
1     8.806452
2     8.472414
3     8.548387
4     7.733333
5     7.254839
6     8.043333
7     5.809677
8     5.935484
9     6.896667
10    8.196774
11    7.606667
12    8.619355
Name: VALOR, dtype: float64

Medidas mensuales de los distintos contaminantes para la estación 4:
MAGNITUD        1         6           7          8           12
MES                                                            
1         8.354839  0.432258   45.516129  42.935484  112.677419
2         8.551724  0.368966   35.586207  37.241379   91.827586
3         8.612903  0.370968   24.483871  40.129032   77.806452
4         8.066667  0.330000   20.233333  37.733333   68.533333
5         7.354839  0.300000   17.193548  36.774194   63.096774
6         7.266667  0.273333   17.800000  40.300000   67.633333
7         7.032258  0.248387   11.225806  41.709677   59.032258
8         7.000000  0.251613    8.225806  37.387097   50.064516
9         8.100000 